In [1]:
# Создание DataFrame и использование join

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import avg, max, asc, desc, col

spark = SparkSession.builder \
    .appName('schema') \
    .config('spark.master', 'spark://spark-master:7077') \
    .getOrCreate()

df1 = spark.createDataFrame([(1, 'Alice'),(2, 'Bob'),(3, 'Den')], ['id', 'name'])
df2 = spark.createDataFrame([(1, 2, 100), (2, 1, 200)], ['id', 'user_id', 'salary'])

df_join = df1.join(df2, df1.id == df2.user_id, how='left').select(df1.name, df2.salary)
df_join.show()

df1.show()

spark.stop()


+-----+------+
| name|salary|
+-----+------+
|Alice|   200|
|  Den|  NULL|
|  Bob|   100|
+-----+------+

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|  Den|
+---+-----+



In [1]:
# Запись данных в Parquet
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('Parquet-HDFS') \
    .config('spark.master', 'spark://spark-master:7077') \
    .config('spark.hadoop.fs.defaultFS', 'hdfs://hadoop-namenode:9000') \
    .getOrCreate()

path = 'hdfs://hadoop-namenode:9000/user/jovyan/parquet'

df1 = spark.createDataFrame([(1, 'Alice'),(2, 'Bob'),(3, 'Den')], ['id', 'name'])
df1.write.format('parquet').mode('overwrite').save(path)

df_parquet = spark.read.parquet(path)
df_parquet.show()

spark.stop()


+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|  Den|
+---+-----+



In [2]:
# Запись данных в ORC
from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder \
    .appName('ORC-HDFS') \
    .config('spark.master', 'spark://spark-master:7077') \
    .config('spark.hadoop.fs.defaultFS', 'hdfs://hadoop-namenode:9000') \
    .getOrCreate()

# Пример данных
data = [("Alice", 25), ("Bob", 30), ("Cathy", 29)]
df = spark.createDataFrame(data, ["Name", "Age"])

path = 'hdfs://hadoop-namenode:9000/user/jovyan/orc'

# Запись данных в ORC
df.write.orc(path)

# Чтение данных из ORC
df_orc = spark.read.orc(path)
df_orc.show()

spark.stop()

+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
|Cathy| 29|
+-----+---+



In [10]:
# Чтение данных с использованием RDD
from pyspark import SparkContext

sc = SparkContext('local', 'Read txt')

rdd = sc.textFile('hdfs://hadoop-namenode:9000/user/jovyan/input/text.txt')

words = rdd.flatMap(lambda line: line.split(' '))
words_counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

print(words_counts.collect())
sc.stop()

[('Заголовок', 1), ('', 2), ('Какой-то', 1), ('текст.', 1), ('-', 2), ('Список', 2)]


In [20]:
# Чтение json файла
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

spark = SparkSession.builder \
    .appName('read json') \
    .config('spark.master', 'spark://spark-master:7077') \
    .config('spark.hadoop.fs.defaultFS', 'hdfs://hadoop-namenode:9000') \
    .getOrCreate()

schema = StructType([
    StructField("users", ArrayType(
        StructType([
            StructField("name", StringType(), True),
            StructField("age", IntegerType(), True)
        ])
    ), True)
])

df_json = spark.read.schema(schema).json("hdfs://hadoop-namenode:9000/user/jovyan/input/test.json")

df_json.show()

# df_users = df.select(explode(col('users')).alias('user'))

# df_final = df_users.select('user.name', 'user.age')
# df_final.show()

spark.stop()

+-----+
|users|
+-----+
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
+-----+



In [27]:
# Чтение csv

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('Read csv') \
    .config('spark.master', 'spark://spark-master:7077') \
    .config('spark.hadoop.fs.defaultFS', 'hdfs://hadoop-namenode:9000') \
    .getOrCreate()

df = spark.read.option('header', 'true').csv('hdfs://hadoop-namenode:9000/user/jovyan/output_csv/')
df.show(truncate=False)

spark.stop()

+-----+---+
|Name |Age|
+-----+---+
|Bob  |20 |
|Ivan |30 |
|Alice|15 |
+-----+---+



In [26]:
# Запись в csv
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('Read csv') \
    .config('spark.master', 'spark://spark-master:7077') \
    .config('spark.hadoop.fs.defaultFS', 'hdfs://hadoop-namenode:9000') \
    .getOrCreate()

data = [('Alice', 15), ('Bob', 20), ('Ivan', 30)]
columns = ['Name', 'Age']

df = spark.createDataFrame(data, columns)
df.write.mode('overwrite').option('header', 'true').csv('hdfs://hadoop-namenode:9000/user/jovyan/output_csv/')

spark.stop()



In [32]:
# Чтение json
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName('Read json sql') \
    .config('spark.master', 'spark://spark-master:7077') \
    .config('spark.hadoop.fs.defaultFS', 'hdfs://hadoop-namenode:9000') \
    .getOrCreate()

path = 'hdfs://hadoop-namenode:9000/user/jovyan/input/test.json'

df = spark.read.json(path)
df.show()

filtered_df = df.filter(col('age') > 30)
filtered_df.show()

grouped_df = df.groupBy('department').agg({"age": "avg", "name": "count"}) \
    .withColumnRenamed("avg(age)", "avg_age") \
    .withColumnRenamed("count(name)", "count")
grouped_df.show()

sorted_df = grouped_df.orderBy(col("count").desc())
sorted_df.show()

sorted_df.write.csv("output.csv", header=True)

spark.stop()

+---------------+----+-----------+-----+
|_corrupt_record| age| department| name|
+---------------+----+-----------+-----+
|              [|NULL|       NULL| NULL|
|           NULL|  30|         HR| John|
|           NULL|  25|    Finance|  Doe|
|           NULL|  35|         HR| Jane|
|           NULL|  40|    Finance| Mark|
|           NULL|  23|Engineering|Smith|
|              ]|NULL|       NULL| NULL|
+---------------+----+-----------+-----+

+---------------+---+----------+----+
|_corrupt_record|age|department|name|
+---------------+---+----------+----+
|           NULL| 35|        HR|Jane|
|           NULL| 40|   Finance|Mark|
+---------------+---+----------+----+

+-----------+-----+-------+
| department|count|avg_age|
+-----------+-----+-------+
|Engineering|    1|   23.0|
|         HR|    2|   32.5|
|       NULL|    0|   NULL|
|    Finance|    2|   32.5|
+-----------+-----+-------+

+-----------+-----+-------+
| department|count|avg_age|
+-----------+-----+-------+
|         

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('SQL') \
    .config('spark.master', 'spark://spark-master:7077') \
    .config('spark.hadoop.fs.defaultFS', 'hdfs://hadoop-namenode:9000') \
    .getOrCreate()

people_df = spark.read.json("hdfs://hadoop-namenode:9000/user/jovyan/input/people.json")
departments_df = spark.read.json("hdfs://hadoop-namenode:9000/user/jovyan/input/depatments.json")

people_df.show()
departments_df.show()

people_df.createOrReplaceTempView('people')
departments_df.createOrReplaceTempView('departments')

join_df = spark.sql("""
SELECT p.name, p.age, d.department_name
FROM people p
JOIN departments d ON p.department_id = d.id
""")

join_df.show()

spark.stop()

+---+-------------+-----+
|age|department_id| name|
+---+-------------+-----+
| 30|            1| John|
| 25|            2|  Doe|
| 35|            1| Jane|
| 40|            2| Mark|
| 23|            3|Smith|
+---+-------------+-----+

+---------------+---+
|department_name| id|
+---------------+---+
|             HR|  1|
|        Finance|  2|
|    Engineering|  3|
+---------------+---+

+-----+---+---------------+
| name|age|department_name|
+-----+---+---------------+
| John| 30|             HR|
|  Doe| 25|        Finance|
| Jane| 35|             HR|
| Mark| 40|        Finance|
|Smith| 23|    Engineering|
+-----+---+---------------+



In [4]:
# Подключение к PostgreSQL
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('PySpark PostgreSQL Connection') \
    .config('spark.master', 'spark://spark-master:7077') \
    .config('spark.hadoop.fs.defaultFS', 'hdfs://hadoop-namenode:9000') \
    .config('spark.jars', 'postgresql-42.7.3.jar') \
    .getOrCreate()

url = 'jdbc:postgresql://postgres:5432/sparkdb'
properties = {
    'user': 'sparkuser',
    'password': 'sparkpass',
    'driver': 'org.postgresql.Driver'
}

df = spark.read.jdbc(url=url, table="employees", properties=properties)
df.show()

df.createOrReplaceTempView('empl')

spark.sql("""
    select *
    from empl e
    where e.salary >= 65000
""").show()

spark.stop()

+---+-------+--------+--------+----------+
| id|   name|position|  salary| hire_date|
+---+-------+--------+--------+----------+
|  1|  Alice|Engineer|75000.00|2021-06-15|
|  2|    Bob| Manager|90000.00|2020-05-01|
|  3|Charlie|      HR|60000.00|2019-04-12|
+---+-------+--------+--------+----------+

+---+-----+--------+--------+----------+
| id| name|position|  salary| hire_date|
+---+-----+--------+--------+----------+
|  1|Alice|Engineer|75000.00|2021-06-15|
|  2|  Bob| Manager|90000.00|2020-05-01|
+---+-----+--------+--------+----------+



In [11]:
# Запись в PostgreSQL
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql import Row
from pyspark.sql.functions import to_date

spark = SparkSession.builder \
    .appName('PySpark PostgreSQL Connection') \
    .config('spark.master', 'spark://spark-master:7077') \
    .config('spark.hadoop.fs.defaultFS', 'hdfs://hadoop-namenode:9000') \
    .config('spark.jars', 'postgresql-42.7.3.jar') \
    .getOrCreate()

data = [
    ("Alice", "Engineer", 75000, "2021-06-15"),
    ("Bob", "Manager", 90000, "2020-05-01"),
    ("Charlie", "HR", 60000, "2019-04-12"),
    ("Diana", "Sales", 50000, "2018-01-25")
]

schema = StructType([
    StructField("name", StringType(), True),
    StructField("position", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("hire_date", StringType(), True)  # сначала строка
])

# columns = ['name', 'position', 'salary', 'hire_date']

df = spark.createDataFrame(data, schema)
df = df.withColumn("hire_date", to_date(df.hire_date, "yyyy-MM-dd"))
df.show()

url = 'jdbc:postgresql://postgres:5432/sparkdb'
properties = {
    'user': 'sparkuser',
    'password': 'sparkpass',
    'driver': 'org.postgresql.Driver'
}

df.write.jdbc(
    url=url,
    table="employees_2",
    mode="append",
    properties=properties
)

print('Добавление новых строк завершено')

spark.stop()

+-------+--------+------+----------+
|   name|position|salary| hire_date|
+-------+--------+------+----------+
|  Alice|Engineer| 75000|2021-06-15|
|    Bob| Manager| 90000|2020-05-01|
|Charlie|      HR| 60000|2019-04-12|
|  Diana|   Sales| 50000|2018-01-25|
+-------+--------+------+----------+

Добавление новых строк завершено
